In [1]:
import os
import json
import pandas as pd
import requests
pd.options.display.max_rows = 200
pd.options.display.max_columns = 100
pd.set_option('display.max_colwidth', -1)

<ipython-input-1-a6c9beac1ab0>:7: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
def getDataframe(filename):
    arr = []
    with open('./data/' + filename) as out:
        for line in out:
            j_content = json.loads(line)
            arr.append(j_content)
    df = pd.DataFrame(arr)
    return df

In [3]:
def mongoIdToId(value):
    return dict(value)["$oid"]

In [4]:
producers = getDataframe('producers.json')
producers["organization_id"] = producers["associatedTo"].apply(mongoIdToId)
producers.head()

,_id,associatedTo,__v,organization_id
0,AFB,{'$oid': '534fffb2a3a7292c64a78123'},0.0,534fffb2a3a7292c64a78123
1,AFB - DR Nouvelle-Aquitaine,{'$oid': '534fffb2a3a7292c64a78123'},0.0,534fffb2a3a7292c64a78123
2,ALTOA,{'$oid': '580fb83a88ee38771813e4cb'},NaN,580fb83a88ee38771813e4cb
3,ANFR,{'$oid': '54256fe588ee380af6717348'},NaN,54256fe588ee380af6717348
4,ARS,{'$oid': '534fffb2a3a7292c64a78123'},0.0,534fffb2a3a7292c64a78123


In [5]:
organizations = getDataframe('organizations.json')
organizations["geop_organization_id"] = organizations["_id"].apply(mongoIdToId)
organizations = organizations.merge(producers, left_on="geop_organization_id", right_on="organization_id")
organizations["geop_organization_name"] = organizations["_id_y"]
organizations = organizations[["geop_organization_id", "geop_organization_name"]]
organizations.head()

,geop_organization_id,geop_organization_name
0,534fff68a3a7292c64a77d8d,Département de Saône-et-Loire
1,534fff68a3a7292c64a77d8d,Département de Saône-et-Loire (71)
2,534fff6aa3a7292c64a77d96,Région Hauts-de-France
3,534fff6ba3a7292c64a77da1,ASPA
4,534fff6ba3a7292c64a77da1,CConseil Départemental du Bas-Rhin (CD67)


In [6]:
organizations['dgv_organization_id'] = organizations['geop_organization_id']

In [7]:
dgv_catalog_orgas = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/b7bbfedc-2448-4135-a6c7-104548d396e7',sep=';')

In [8]:
dgv_catalog_orgas = dgv_catalog_orgas[['id','name', 'metric.datasets']]
dgv_catalog_orgas.head()

,id,name,metric.datasets
0,624c5cdc8eccbf6500771af5,PeopleSpheres,0
1,624c54d28c29de3315bd5984,Défi iDoc Santé,0
2,624afc9ad677e79b461b72c6,Upenergie,0
3,624ae80064cf94e7bce49ab6,Clean Hydrogen Partnership,0
4,6248b0fbcc73e9eaac126843,FLEUR DES ILES MAYOTTE,0


In [9]:
dgv_catalog_orgas = dgv_catalog_orgas.rename(columns={
    'id': 'dgv_organization_id',
    'name': 'dgv_organization_name', 
    'metric.datasets': 'dgv_nb_datasets',
})

In [10]:
# no duplicates
assert organizations[organizations.duplicated()].empty
assert dgv_catalog_orgas[dgv_catalog_orgas.duplicated()].empty

In [11]:
organizations.columns

Index(['geop_organization_id', 'geop_organization_name',
       'dgv_organization_id'],
      dtype='object')

In [12]:
dgv_catalog_orgas.dgv_organization_id.nunique()

3341

In [13]:
# outer merge to pivot on dgv orgas, not necessarily on geop orgs
final_orgas = dgv_catalog_orgas.merge(organizations, on='dgv_organization_id', how='outer')
assert final_orgas[final_orgas.duplicated()].empty
final_orgas.head()

,dgv_organization_id,dgv_organization_name,dgv_nb_datasets,geop_organization_id,geop_organization_name
0,624c5cdc8eccbf6500771af5,PeopleSpheres,0.0,NaN,NaN
1,624c54d28c29de3315bd5984,Défi iDoc Santé,0.0,NaN,NaN
2,624afc9ad677e79b461b72c6,Upenergie,0.0,NaN,NaN
3,624ae80064cf94e7bce49ab6,Clean Hydrogen Partnership,0.0,NaN,NaN
4,6248b0fbcc73e9eaac126843,FLEUR DES ILES MAYOTTE,0.0,NaN,NaN


In [52]:
# geoc = pd.read_excel('./data/Liste_catalogues_DCAT.xlsx')
geoc = pd.read_excel('./data/Liste_producteur_fevrier_avec_ressources.xlsx')
del geoc["Unnamed: 3"]

In [53]:
geoc = geoc.rename(columns={
    'Producteurs': 'geoc_organization_name', 
    'uuid': 'geoc_organization_uuid', 
    'url de moissonnage ': 'geoc_organization_url'
})
geoc.head()

,geoc_organization_name,geoc_organization_uuid,geoc_organization_url
0,Agence de l'eau Adour Garonne (AEAG),bdb31aaf37e0f29db345d2b62d7ccccb,http://www.geocatalogue.fr/www/dcat/bdb31aaf37e0f29db345d2b62d7ccccb
1,"Agence nationale de sécurité sanitaire de l'alimentation, de l'environnement et du travail (ANSES)",cadef4c11943e372e8afd0b94797b5ec,http://www.geocatalogue.fr/www/dcat/cadef4c11943e372e8afd0b94797b5ec
2,Agence Technique de l'information sur l'hospitalisation (ATIH),98283bf812e489423f273f61a4e4e431,http://www.geocatalogue.fr/www/dcat/98283bf812e489423f273f61a4e4e431
3,Agglomération du Choletais,e05f395b2c2b3a94fabd65275d6ae266,http://www.geocatalogue.fr/www/dcat/e05f395b2c2b3a94fabd65275d6ae266
4,Air Pays de la Loire,f540ee38176857fd4ed78c5c321b9ac0,http://www.geocatalogue.fr/www/dcat/f540ee38176857fd4ed78c5c321b9ac0


In [54]:
def removeCommonWords(org):
    org = str(org)
    org = org.replace(' de ',' ').replace(' un ',' ').replace(' à ','').replace(' l\'',' ').replace(' d\'', ' ')
    org = org.replace(' des ',' ').replace(' pour ', ' ').replace(' les ', ' ').replace('(','').replace(')','')
    org = org.replace(' - ',' ').replace(',','').replace(' du ', ' ').replace('à','a').replace('é','e')
    org = org.replace('ê','e').replace('è','e').replace('\'',' ')
    return org.lower()

In [55]:
geoc["geoc_slug"] = geoc["geoc_organization_name"].apply(removeCommonWords)
final_orgas["geop_slug"] = final_orgas["geop_organization_name"].apply(removeCommonWords)
final_orgas["dgv_slug"] = final_orgas["dgv_organization_name"].apply(removeCommonWords)

In [56]:
df_dgv_match = final_orgas.merge(geoc, left_on="dgv_slug", right_on="geoc_slug")
df_dgv_match["geoc_organization_url"].drop_duplicates().shape

(98,)

In [57]:
df_geop_match = final_orgas.merge(geoc, left_on="geop_slug", right_on="geoc_slug")
df_geop_match["geoc_organization_url"].drop_duplicates().shape

(33,)

In [58]:
df_final = df_dgv_match.merge(df_geop_match, how="outer")
df_final[df_final['geoc_organization_uuid'].notna()]["geoc_organization_url"].drop_duplicates().shape

(121,)

In [59]:
df_final[df_final['geoc_organization_uuid'].notna()].head()

,dgv_organization_id,dgv_organization_name,dgv_nb_datasets,geop_organization_id,geop_organization_name,geop_slug,dgv_slug,geoc_organization_name,geoc_organization_uuid,geoc_organization_url,geoc_slug
0,62287c5b201dffeb66cc385a,BRL,3.0,NaN,NaN,nan,brl,BRL,1a0b4aa361bb5315e304831a320f89b3,http://www.geocatalogue.fr/www/dcat/1a0b4aa361bb5315e304831a320f89b3,brl
1,61fc10b5285e11c883d018bd,MAIRIE LES TROIS DOMAINES,0.0,NaN,NaN,nan,mairie les trois domaines,Mairie de Les Trois domaines,6fc21c45008161fcadd99a9c53e0351f,http://www.geocatalogue.fr/www/dcat/6fc21c45008161fcadd99a9c53e0351f,mairie les trois domaines
2,61b0b0673c11d387da1df057,Pontivy Communaute,0.0,NaN,NaN,nan,pontivy communaute,Pontivy Communauté,eac49cc9bdb321a31f6150862adc28b7,http://www.geocatalogue.fr/www/dcat/eac49cc9bdb321a31f6150862adc28b7,pontivy communaute
3,61484aaaffc949c8b4971434,Préfecture de la région Hauts-de-France,1.0,NaN,NaN,nan,prefecture la region hauts-de-france,Préfecture de la région Hauts-de-France,0709d43ca86491e10b8d95b9ae7ad916,http://www.geocatalogue.fr/www/dcat/0709d43ca86491e10b8d95b9ae7ad916,prefecture la region hauts-de-france
4,61322119e8f2ff4e8225c747,Préfecture des Landes,0.0,NaN,NaN,nan,prefecture landes,Préfecture des Landes,81bbfc2b3e0c3589fb5f868f157f0f65,http://www.geocatalogue.fr/www/dcat/81bbfc2b3e0c3589fb5f868f157f0f65,prefecture landes


In [60]:
df_final.to_csv('mapping_geop_dgv_geoc-pivot-datagouv-v2.csv', index=False)

In [61]:
df_final.shape

(172, 11)

In [27]:
from collections import defaultdict

url = "https://www.data.gouv.fr/api/1"
auth = {
    "x-api-key": os.getenv("DATAGOUV_TOKEN")
}

def get_org_data(org_id):
    org_data = defaultdict(list)
    org_data["id"] = org_id
    r = requests.get(f"{url}/organizations/{org_id}/")
    data = r.json()

    # members
    for m in data["members"]:
        r = requests.get(m["user"]["uri"], headers=auth)
        mdata = r.json()
        org_data[m["role"]].append(f"{mdata['first_name']} {mdata['last_name']} <{mdata['email']}>")
    
    org_data["admin"] = ", ".join(org_data["admin"])
    org_data["editor"] = ", ".join(org_data["editor"])

    # datasets
    r = requests.get(f"{url}/organizations/{org_id}/datasets/?page_size=1000")
    datasets = r.json()
    assert datasets["total"] == len(datasets["data"])
    org_data["nb_datasets"] = datasets["total"]
    org_data["nb_datasets_geop"] = len([d["id"] for d in datasets["data"] if "geop:dataset_id" in d["extras"]])
    org_data["last_dataset_modified_date"] = max([d["last_modified"] for d in datasets["data"]]) if datasets["data"] else None
        
    return dict(org_data)

In [28]:
metadata = []
for org_id in df_final["dgv_organization_id"].drop_duplicates():
    metadata.append(get_org_data(org_id))

In [62]:
del df_final["dgv_nb_datasets"]

In [33]:
r = requests.get(f"{url}/harvest/sources/?page_size=1000", headers=auth)
harvesters = r.json()
assert harvesters["total"] == len(harvesters["data"])
len(harvesters["data"])

272

In [66]:
# harvesters
def get_harvest_data(org_id):
    hs = [h for h in harvesters["data"] if h["organization"] and h["organization"].get("id") == org_id]
    return {
        "id": org_id,
        "nb_harvesters": len(hs),
        "harvesters": " | ".join([
            f"{h['id']} — {h['name']} — {h['url']}" for h in hs
        ])
    }

harvesters_match = []
for org_id in df_final["dgv_organization_id"].drop_duplicates():
    harvesters_match.append(get_harvest_data(org_id))
    
len([h for h in harvesters_match if h['nb_harvesters']])

20

In [67]:
meta_df = pd.DataFrame(metadata)
harvest_df = pd.DataFrame(harvesters_match)
df_meta = df_final.merge(meta_df, right_on="id", left_on="dgv_organization_id")
del df_meta["id"]
df_meta = df_meta.merge(harvest_df, right_on="id", left_on="dgv_organization_id")
del df_meta["id"]
df_meta

,dgv_organization_id,dgv_organization_name,geop_organization_id,geop_organization_name,geop_slug,dgv_slug,geoc_organization_name,geoc_organization_uuid,geoc_organization_url,geoc_slug,admin,editor,nb_datasets,nb_datasets_geop,last_dataset_modified_date,nb_harvesters,harvesters
0,62287c5b201dffeb66cc385a,BRL,NaN,NaN,nan,brl,BRL,1a0b4aa361bb5315e304831a320f89b3,http://www.geocatalogue.fr/www/dcat/1a0b4aa361bb5315e304831a320f89b3,brl,BRL MARSAL <fabrice.marsal@brl.fr>,,3,0,2022-03-29T14:56:48.848000,0,
1,61fc10b5285e11c883d018bd,MAIRIE LES TROIS DOMAINES,NaN,NaN,nan,mairie les trois domaines,Mairie de Les Trois domaines,6fc21c45008161fcadd99a9c53e0351f,http://www.geocatalogue.fr/www/dcat/6fc21c45008161fcadd99a9c53e0351f,mairie les trois domaines,Christian BAZART <les.trois.domaines@ozone.net>,,0,0,None,0,
2,61b0b0673c11d387da1df057,Pontivy Communaute,NaN,NaN,nan,pontivy communaute,Pontivy Communauté,eac49cc9bdb321a31f6150862adc28b7,http://www.geocatalogue.fr/www/dcat/eac49cc9bdb321a31f6150862adc28b7,pontivy communaute,Bernard Le Breton <opendata@pontivy-communaute.bzh>,,1,0,2021-12-08T14:21:11.234000,0,
3,61484aaaffc949c8b4971434,Préfecture de la région Hauts-de-France,NaN,NaN,nan,prefecture la region hauts-de-france,Préfecture de la région Hauts-de-France,0709d43ca86491e10b8d95b9ae7ad916,http://www.geocatalogue.fr/www/dcat/0709d43ca86491e10b8d95b9ae7ad916,prefecture la region hauts-de-france,Sébastien MUHLEBACH <sebastien.muhlebach@hauts-de-france.gouv.fr>,,1,0,2022-03-29T08:51:04.919000,0,
4,61322119e8f2ff4e8225c747,Préfecture des Landes,NaN,NaN,nan,prefecture landes,Préfecture des Landes,81bbfc2b3e0c3589fb5f868f157f0f65,http://www.geocatalogue.fr/www/dcat/81bbfc2b3e0c3589fb5f868f157f0f65,prefecture landes,Frédéric DUBERT <frederic.dubert@developpement-durable.gouv.fr>,,0,0,None,0,
5,60ba2f3eb3735a080f1fbda7,Communauté de Communes de Haute Cornouaille,NaN,NaN,nan,communaute communes haute cornouaille,Communauté de Communes de Haute Cornouaille,1f9c319f589bbea6f8bbf65bcf6be730,http://www.geocatalogue.fr/www/dcat/1f9c319f589bbea6f8bbf65bcf6be730,communaute communes haute cornouaille,Jean-François NIGER <sig@haute-cornouaille.fr>,,16,0,2022-03-25T09:10:25.030000,0,
6,60b887112966e85ad4af5d87,Vendée Eau,NaN,NaN,nan,vendee eau,Vendée Eau,e025e20163c523060771f2d7b4975533,http://www.geocatalogue.fr/www/dcat/e025e20163c523060771f2d7b4975533,vendee eau,Laura Lecoeur <laura.lecoeur@vendee-eau.fr>,,1,0,2021-06-03T09:55:04.280000,0,
7,60af47590c0f0c2ad4d2b3db,GEOPAL,NaN,NaN,nan,geopal,GéoPAL,bab36dee9674dec90cc6a25bcbbce2be,http://www.geocatalogue.fr/www/dcat/bab36dee9674dec90cc6a25bcbbce2be,geopal,"Jérôme AUBRET <jerome.aubret@gigalis.fr>, Martin Delsinne <martin.delsinne@geovendee.fr>",,1,0,2022-03-03T09:29:48.114000,0,
8,60ae1a87d2b49820073b1a8b,GRAND ANNECY Agglomération,NaN,NaN,nan,grand annecy agglomeration,Grand Annecy Agglomération,a9a1bda97fbc95346022044faecf4811,http://www.geocatalogue.fr/www/dcat/a9a1bda97fbc95346022044faecf4811,grand annecy agglomeration,"Guillaume PONZA <gponza@grandannecy.fr>, DSI GRAND ANNECY <informatique@grandannecy.fr>, Domitille GOMBERT <dgombert@grandannecy.fr>",,6,0,2021-08-16T11:29:57.517000,0,
9,60a3c84131bef0b3b4ef4b98,TIGEO,NaN,NaN,nan,tigeo,Tigeo,5a33e14720926d8278259f2ed7db2958,http://www.geocatalogue.fr/www/dcat/5a33e14720926d8278259f2ed7db2958,tigeo,Alice Martin <alice.martin@tigeo.fr>,,14,0,2022-03-31T11:03:27.169000,0,


In [68]:
df_meta.to_csv('mapping_geop_dgv_geoc-pivot-datagouv_w_metadata_harvest.csv',index=False)

In [26]:
# TODO:
# - [x] add nb of datasets
# - [x] add nb of geop datasets
# - [x] add admin contacts
# - [x] add members contacts
# - [x] last modified dataset date
# - [x] match w/o joining on geop_organizations (dgv_organizations)